In [26]:
library(ggplot2)
library(ggExtra)
library(brms)
library(reshape2)
library(coda)
library(tidybayes)
library(ggstance)
library(viridis)
library(latex2exp)
library(ggthemes)
library(data.table)

In [27]:
get_data <- function(){
    filename <- "C:/Users/Arkady/Google Drive/data/beyond_the_reach/k-values.txt"
    data <- read.table(filename, header = TRUE, sep = "\t")
    data[, 'subj_id'] <- factor(data[, 'subj_id'])  
    data[, 'task'] <- factor(data[, 'task'])  
    data[, 'order'] <- factor(data[, 'order'])  
    return(data)
}

data <- get_data()

In [33]:
get_bf_k <- function(data){
    # sd(IV) in this case is 0.5, as IV is task (equal number of 0's and 1's)
    priors_task <- c(set_prior(sprintf('normal(%f, %f)', mean(data$k), sd(data$k)), class = 'Intercept'),
                    set_prior(sprintf('cauchy(0.0, %f)', 0.707*sd(data$k)/0.5), class = 'b'))

    m_null <- brm(k ~ (1 | subj_id), data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task[1,],
                  refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))
    m_task <- brm(k ~ (1 | subj_id) + task, data=data, family=gaussian(), save_all_pars=TRUE, prior=priors_task, 
                  refresh=0, control = list(adapt_delta = 0.9, max_treedepth=15))
    bf <- bayes_factor(x1=m_task, x2=m_null)$bf
    names(bf) <- 'bf'
    
    # just in case, we also record the 0.95-highest density interval
    hpd<-HPDinterval(as.mcmc(m_task, combine_chains = TRUE))['b_taskwalking',]
    
    result = list(bf=t(c(bf, hpd)), m_null=m_null, m_task=m_task)
    
    return(result)
}

In [36]:
result <- get_bf_k(data)

Compiling the C++ model
Start sampling



Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.301 seconds (Warm-up)
               0.15 seconds (Sampling)
               0.451 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.33 seconds (Warm-up)
               0.105 seconds (Sampling)
               0.435 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.269 seconds (Warm-up)
               0.132 seconds (Sampling)
               0.401 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.24 seconds (Wa

Compiling the C++ model
Start sampling



Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.294 seconds (Warm-up)
               0.17 seconds (Sampling)
               0.464 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.314 seconds (Warm-up)
               0.151 seconds (Sampling)
               0.465 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.297 seconds (Warm-up)
               0.143 seconds (Sampling)
               0.44 seconds (Total)


Gradient evaluation took 0 seconds
1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Adjust your expectations accordingly!



 Elapsed Time: 0.276 seconds (W

In [42]:
result$bf

bf,
0.1637817,0


In [40]:
hpd<-HPDinterval(as.mcmc(result$m_task, combine_chains = TRUE))['b_taskwalking',]

In [41]:
hpd

lower       upper 
-0.08676431  0.02042039